In [ ]:
#Payroll_State_Report.py

import sys
from rms_roa_cmn.utils import Utils
from rms_roa_cmn.mail import Mail
from rms_roa_cmn.enums import FileType, LogLevel
from Repository import Repository
import os
import glob

def main():
    utl = Utils()
    mail = Mail()
    db = Repository()
    try:
        utl.Log('Start: Payroll State Report.', LogLevel.INFO)
        #param = argv[1]
        #moveFile = 'Y' if (len(sys.argv) > 1 and sys.argv[1] == 'Y') else 'N'
        param = int(sys.argv[1]) if (len(sys.argv) > 1) else 0
        fiscalFlag = utl.GetConfig('fiscalFlag')
        #folders
        payrollPath = utl.GetConfig('payrollPath')
        yrFolderNm = utl.GetConfig('yrFolderNm')
        yrQtrFolderNm = utl.GetConfig('yrQtrFolderNm')
        archiveFolderNm = utl.GetConfig('archiveFolderNm')
        rawDataFolderNm = utl.GetConfig('rawDataFolderNm')
        filesToRunFolderNm = utl.GetConfig('files_to_run')
        payrollDataFolderNm = utl.GetConfig('payroll_data')
        #PR
        #table_PR = utl.GetConfig('table_PR')
        importQryPR = utl.GetConfig('importQryPR')
        #CC
        importQryCC = utl.GetConfig('importQryCC')
        importQryWT = utl.GetConfig('importQryWT')
        #table_CC_wt = utl.GetConfig('table_CC_wt')

        #wt_start_col = utl.GetConfig('wt_start_col')
        sqlQryProcess = utl.GetConfig('sqlQryProcess')
        sqlQryInitializeData = utl.GetConfig('sqlQryInitializeData')
        bad_rpt_ids = utl.GetConfig('bad_rpt_ids').split('|')
        bad_rpt_descs = utl.GetConfig('bad_rpt_descs').split('|')
        output_CCRecon_file = utl.GetConfig('output_CCRecon_file')
        output_NCCI_Dtl_file = utl.GetConfig('output_NCCI_Dtl_file')
        NCCI_listing = utl.GetConfig('NCCI_listing')
        NCCI_listing_SAS_ws = utl.GetConfig('NCCI_listing_SAS_ws')
        NCCI_importQry = utl.GetConfig('NCCI_importQry')
        #NCCI_Codes_Master = utl.GetConfig('NCCI_Codes_Master')
        rpt_sql_qry = utl.GetConfig('rpt_sql_qry')
        exception_rpt_file = utl.GetConfig('exception_rpt_file')
        generic_positions_file = utl.GetConfig('generic_positions_file')
        sap_generic_role_details = utl.GetConfig('sap_generic_role_details')
        sap_gen_role_dtl_ws = utl.GetConfig('sap_gen_role_dtl_ws')
        NCCI_Dtl_importQry = utl.GetConfig('NCCI_Dtl_importQry')
        NCCI_Dtl_gen_only_ws = utl.GetConfig('NCCI_Dtl_gen_only_ws')
        #NCCI_Dtl_file = utl.GetConfig('NCCI_Dtl_file')
        GENERIC_Role_importQry = utl.GetConfig('GENERIC_Role_importQry')
        #GENERIC_Role_file = utl.GetConfig('GENERIC_Role_file')
        sqlQry_NCCI_Generic = utl.GetConfig('sqlQry_NCCI_Generic')
        Link_NCCI_Detail = utl.GetConfig('Link_NCCI_Detail')
        FINAL_NCCI_Detail = utl.GetConfig('FINAL_NCCI_Detail')
        emailSubj = utl.GetConfig('email_subject')

        utl.Log('Initialize/Reset data.', LogLevel.INFO)
        result = db.Initialize_Data(sqlQryInitializeData, param)
        if fiscalFlag == 'Y':
            yr = result[0][2]
            qtr = result[0][3]
            yrPrefix = 'F'
        else:
            yr = result[0][0]
            qtr = result[0][1]
            yrPrefix = 'C'

        yr_qtr = r'{}Y{} Q{}'.format(yrPrefix, yr, qtr)
        payrollDataFolderNm = payrollDataFolderNm.format(qtr, yrPrefix, yr)

        utl.Log('Create Year folder', LogLevel.INFO)
        yrPath = db.Create_Folder(r'{}{}'.format(payrollPath, yrFolderNm.format(yr)))
        utl.Log('Create Year-Quarter folder', LogLevel.INFO)
        yrQtrPath = db.Create_Folder(r'{}\{}'.format(yrPath, yrQtrFolderNm.format(yrPrefix, yr, qtr)))
        utl.Log('Create Archive folder', LogLevel.INFO)
        archivePath = db.Create_Folder(r'{}\{}'.format(yrQtrPath, archiveFolderNm))
        utl.Log('Create Raw data folder', LogLevel.INFO)
        rawDataPath = db.Create_Folder(r'{}\{}'.format(yrQtrPath, rawDataFolderNm))
        utl.Log('Create Files to be run folder', LogLevel.INFO)
        filesToRunPath = db.Create_Folder(r'{}\{}'.format(rawDataPath, filesToRunFolderNm))
        utl.Log('Create Payroll_Data folder folder', LogLevel.INFO)
        payrollDataPath = db.Create_Folder(r'{}\{}'.format(rawDataPath, payrollDataFolderNm))

        output_CCRecon_filePath = r'{}\{}'.format(yrQtrPath, output_CCRecon_file.format(yr_qtr))
        output_NCCI_Dtl_filePath_1 = r'{}\{}'.format(payrollPath, output_NCCI_Dtl_file.format(yr_qtr))
        output_NCCI_Dtl_filePath_2 = r'{}\{}'.format(yrQtrPath, output_NCCI_Dtl_file.format(yr_qtr))
        NCCI_listing_filePath = r'{}{}'.format(payrollPath, NCCI_listing.format(yr))
        exception_rpt_file = exception_rpt_file.format(yr_qtr)
        exception_rpt_filePath = r'{}\{}'.format(yrQtrPath, exception_rpt_file)
        generic_positions_filePath = r'{}\{}'.format(archivePath, utl.SetFileDateStamp(generic_positions_file, utl.CurrentDt))
        sap_generic_role_details_filePath = r'{}\{}'.format(yrQtrPath, sap_generic_role_details.format(yr_qtr))
        #NCCI_detail_filePath = r'{}\{}'.format(yrPath, NCCI_Dtl_file.format(yr))
        #GENERIC_Role_filePath = r'{}\{}'.format(yrPath, GENERIC_Role_file.format(yr))
        Link_NCCI_Detail_filePath = r'{}\{}'.format(yrQtrPath, Link_NCCI_Detail.format(yr_qtr))
        FINAL_NCCI_Detail_filePath = r'{}\{}'.format(yrQtrPath, FINAL_NCCI_Detail.format(yr_qtr))
        
        param = 1
        
        if param == 1 or param == 2:
            if param == 1:
                utl.Log('Extract zip files.', LogLevel.INFO)
                fileCount = db.Extract_Files('{}{}'.format(payrollPath, '*.zip'), payrollDataPath)
                utl.Log('#files extracted: {}'.format(fileCount), LogLevel.INFO)

                utl.Log('Import RAW CC/PR files.', LogLevel.INFO)
                
                # 1) See the top-level directories (and subfolders) inside payrollDataPath
                utl.Log(f"payrollDataPath: {payrollDataPath}", LogLevel.INFO)
                utl.Log(f"Top-level items after unzip: {os.listdir(payrollDataPath)}", LogLevel.INFO)

                # 2) Show everything that matches your pattern
                pattern_pr = os.path.join(payrollDataPath, "PR*.xlsx")  # or whatever you�re using
                pattern_cc = os.path.join(payrollDataPath, "New CC*.xlsx")
                pr_found = glob.glob(pattern_pr)
                cc_found = glob.glob(pattern_cc)

                utl.Log(f"PR pattern = {pattern_pr}", LogLevel.INFO)
                utl.Log(f"PR matched = {pr_found}", LogLevel.INFO)
                utl.Log(f"CC pattern = {pattern_cc}", LogLevel.INFO)
                utl.Log(f"CC matched = {cc_found}", LogLevel.INFO)
                
                
                #PR
                prFiles = db.Load_Raw_PR_Data(r'{}\{}'.format(payrollDataPath, 'PR*.xlsx'), yr, qtr, importQryPR)
                utl.Log('#PR files imported: {}'.format(len(prFiles)), LogLevel.INFO)
                utl.Log('Copy PR files to be run folder.', LogLevel.INFO)
                db.Copy_Files(prFiles, filesToRunPath)

                #CC
                #ccFiles = db.Load_Raw_CC_Data(r'{}\{}'.format(payrollDataPath, '*CC*.xls'), importQryCC, yr, qtr, table_CC_wt)
                ccFiles = db.Load_Raw_CC_Data(r'{}\{}'.format(payrollDataPath, 'New CC*.xlsx'), importQryCC, yr, qtr, importQryWT)
                utl.Log('#CC files imported: {}'.format(len(ccFiles)), LogLevel.INFO)
                utl.Log('Copy CC files to be run folder.', LogLevel.INFO)
                db.Copy_Files(ccFiles, filesToRunPath)

            #import NCCI listing`
            result = db.Load_NCCI_Listing(NCCI_listing_filePath, NCCI_listing_SAS_ws, NCCI_importQry, yr, qtr)
            utl.Log('#NCCI listing records: {}'.format(result), LogLevel.INFO)

            #process RAW BW-Old CC/CC-Old PR files
            utl.Log('Process RAW CC/PR files.', LogLevel.INFO)
            result = db.Process_Raw_Data(sqlQryProcess)
            utl.Log('Raw files have been processed.', LogLevel.INFO)

            utl.Log('Export BAD reports', LogLevel.INFO)
            i = 0
            for rpt_id in bad_rpt_ids:
                utl.Log(bad_rpt_descs[i], LogLevel.INFO)
                df = db.Get_Report_Data(rpt_sql_qry, [rpt_id])
                fMode = 'w' if (i == 0) else 'a'
                utl.ExportToFile(df, exception_rpt_filePath, FileType.Excel, fileMode=fMode, worksheetName=bad_rpt_descs[i], setColWidth='Y')
                i += 1

            utl.Log('Export Output NCCI Detail', LogLevel.INFO)
            df = db.Get_Report_Data(rpt_sql_qry, [2])
            utl.ExportToFile(df, output_NCCI_Dtl_filePath_1, FileType.Excel, fileMode='w', worksheetName='All Data', setColWidth='Y')

            utl.Log('Export Output CC Recon', LogLevel.INFO)
            df = db.Get_Report_Data(rpt_sql_qry, [8])
            utl.ExportToFile(df, output_CCRecon_filePath, FileType.Excel, fileMode='w', worksheetName='Sheet1', setColWidth='Y')

            utl.Log('Pull Quarterly Data for Generic Positions', LogLevel.INFO)
            df = db.Get_Report_Data(rpt_sql_qry, [9])
            utl.ExportToFile(df, generic_positions_filePath, FileType.CSV)
            utl.ExportToFile(df, sap_generic_role_details_filePath, FileType.Excel, fileMode='w', worksheetName=sap_gen_role_dtl_ws, setColWidth='Y')

            utl.Log('Export "Output NCCI Detail" for the Executive Cap Calculations and ultimately your State Report ', LogLevel.INFO)
            df = db.Get_Report_Data(rpt_sql_qry, [5])
            utl.ExportToFile(df, output_NCCI_Dtl_filePath_2, FileType.Excel, fileMode='w', worksheetName='All Data', setColWidth='Y')
        elif param == 3:
            utl.Log('Calculate Percentages for NCCI Detail File', LogLevel.INFO)

            #import NCCI details
            utl.Log('Import "Output NCCI detail file"', LogLevel.INFO)
            result = db.Load_NCCI_Details(output_NCCI_Dtl_filePath_2, NCCI_Dtl_gen_only_ws, NCCI_Dtl_importQry)
            utl.Log('#NCCI detail records: {}'.format(result), LogLevel.INFO)

            #GENERIC Roles
            utl.Log('Import GENERIC Roles file', LogLevel.INFO)
            result = db.Load_Generic_Roles(sap_generic_role_details_filePath, sap_gen_role_dtl_ws, GENERIC_Role_importQry)
            utl.Log('#GENERIC Roles: {}'.format(result), LogLevel.INFO)

            utl.Log('Run Payroll #1', LogLevel.INFO)
            df = db.Get_Report_Data(sqlQry_NCCI_Generic, [1])
            utl.ExportToFile(df, Link_NCCI_Detail_filePath, FileType.Excel, fileMode='w', setColWidth='Y')
        elif param == 4:
            #RE-IMPORT GENERIC Roles
            utl.Log('Re-Import GENERIC Roles file', LogLevel.INFO)
            result = db.Load_Generic_Roles(sap_generic_role_details_filePath, sap_gen_role_dtl_ws, GENERIC_Role_importQry)
            utl.Log('#GENERIC Roles: {}'.format(result), LogLevel.INFO)

            utl.Log('Run Payroll #2', LogLevel.INFO)
            df = db.Get_Report_Data(sqlQry_NCCI_Generic, [2])
            utl.ExportToFile(df, FINAL_NCCI_Detail_filePath, FileType.Excel, fileMode='w', setColWidth='Y')
        elif param == 0:
            raise Exception('ERROR: Application parameter is MISSING!')

        utl.Log('Send email STEP {}.'.format(param), LogLevel.INFO)
        if utl.GetConfig('email_enabled') == 'Y':
            mail.Send(subj=emailSubj.format(param))
        utl.Log('End STEP {}.'.format(param), LogLevel.INFO)
    except KeyboardInterrupt:
        pass
    except:
        errorMsg = utl.ErrorHandler()
        utl.Log(errorMsg, LogLevel.ERROR)
        if utl.GetConfig('email_enabled') == 'Y':
            mail.Send(msg=errorMsg, exception='Y')

if __name__ == "__main__":
    main()

